In [1]:
import numpy as np
import os
import subprocess
import gzip

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv1D, AveragePooling1D, MaxPooling1D, TimeDistributed, LeakyReLU, BatchNormalization, Flatten
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras import regularizers

import keras
from keras.models import Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate, Conv1D, BatchNormalization, Flatten
from keras.optimizers import Adagrad, Adam
from keras.utils import plot_model
from keras import regularizers

from keras.utils import plot_model
import pickle
from keras.models import load_model

import numpy as np
from keras.models import load_model
from sklearn.model_selection import KFold
from time import time
from timeit import default_timer as timer
import pickle
import gzip



Using TensorFlow backend.


In [ ]:
dataset = np.load('all_dataset_window19MiddleQ2.npy') # load

In [24]:
f = gzip.GzipFile('all_dataset_window9RightSideQ8.npy.gz', "r")
dataset = np.load(f)

In [25]:
print(dataset.shape)

(1161181, 9, 50)


In [ ]:
f = gzip.GzipFile('all_dataset_window19Middle.npy.gz', "w")
np.save(f, dataset)
f.close()

In [26]:
windowSize = 9
#predictionIndex = 9
classSize = 8  # 2 or 3 
#numberOfFeatures = 50 #44 

amino_acid_residues = 21

In [27]:
LR = 0.0005
drop_out = 0.3
batch_dim = 64
nn_epochs = 5
w_reg = regularizers.l2(0.0001)
number_filters = 16


def get_model():
   
    LR = 0.0005
    drop_out = 0.3
    batch_dim = 64
    nn_epochs = 5
    w_reg = regularizers.l2(0.0001)
    number_filters = 16

    loss = 'categorical_crossentropy'


    input_shape = (windowSize, 21)

    conv1_input = Input(shape=(windowSize, 21), name='InputWindow')

    conv_1 = Conv1D( 64, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg)(conv1_input)
    conv_1 = BatchNormalization(name='BN1')(conv_1)
    conv_2 = Conv1D( 64, 3, strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_1)
    conv_2 = BatchNormalization(name='BN2')(conv_2)
    conv_3 = Conv1D( 64, 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_2)
    conv_3 = BatchNormalization(name='BN3')(conv_3)



    conv_4 = Conv1D( 64, 3, strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg)(conv_3)
    conv_4 = BatchNormalization(name='BN4')(conv_4)
    conv_5 = Conv1D( 64, 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_4)
    conv_5 = BatchNormalization(name='BN5')(conv_5)
    #conv_6 = Conv1D( 73, 3, strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_5)
    #conv_6 = BatchNormalization(name='BN6')(conv_6)


    #conv_7 = Conv1D( 73, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg)(conv_6)
    #conv_7 = BatchNormalization(name='BN7')(conv_7)
    #conv_8 = Conv1D( 4, 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_7)
    #conv_8 = BatchNormalization(name='BN8')(conv_8)
    #conv_9 = Conv1D( 4, 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_8)
    #conv_9 = BatchNormalization(name='BN9')(conv_9)


    flatten  = Flatten()(conv_5)
    first_dense = Dense(16, activation='relu', use_bias=True,  kernel_regularizer=w_reg, name='last')(flatten)
    first_dense = BatchNormalization(name='BN10')(first_dense)
    final_model_output = Dense(classSize, activation = 'softmax', name='softmax')(first_dense)

    m = Model(inputs=conv1_input, outputs=final_model_output)

    opt = Adam(lr=LR)
    m.compile(optimizer=opt, loss=loss,metrics=['accuracy', 'mae'])

    print("\nHyper Parameters\n")
    print("Learning Rate: " + str(LR))
    print("Drop out: " + str(drop_out))
    print("Batch dim: " + str(batch_dim))
    print("Number of epochs: " + str(nn_epochs))
    print("Regularizers: " + str(w_reg.l2))
    print("\nLoss: " + loss + "\n")
    m.summary()

    #import os
    #os.environ["PATH"] += os.pathsep + 'C:/Users/Ieremie/Anaconda3/pkgs/graphviz-2.38-hfd603c8_2/Library/bin'

    #plot_model(m)#, to_file='model.png')
    return m
    

In [28]:
from sklearn.utils import shuffle
dataset = shuffle(dataset, random_state=111)
print(dataset.shape)
print(dataset[0,:2,:])

(1161181, 9, 50)
[[0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.         0.         0.41095954
  0.19940776 0.02436359 0.4750208  0.07585818 0.0099518  0.90378445
  0.80059224 0.09708863 0.63876313 0.3015348  0.03106848 0.15842418
  0.99356788 0.14430314 0.09621554 0.12786157 0.94213307 0.00966052
  0.26894143 0.04653048]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.         0.         0.6942364
  0.01365366 0.62010646 0.39174098 0.02436359 0.05215356 0.15709548
  0.15577585 0.93213767 0.08166025 0.08471056 0.20100899 0.0295981
  0.

In [ ]:
kf = KFold(n_splits=10, random_state=10, shuffle=True)
for train, test in kf.split(dataset):
    print(train,test)

In [29]:
l = 0
r = 113350
n = 1133502

fold = 0
valid_indexes = [()]


for i in range (10):
  
    fold += 1
    print(fold)
    
    m = get_model()
    start_time = timer()
    history = m.fit(np.concatenate((dataset[0:l], dataset[r:n]), axis=0)[:,:,21+classSize:], np.concatenate((dataset[0:l], dataset[r:n]), axis=0)[:, 0, 21 : (21+classSize)],
                    epochs=nn_epochs, validation_data=(dataset[l:r][:,:,21+classSize:], dataset[l:r][:, 0, 21 : (21+classSize)]) ,shuffle=True,batch_size=batch_dim)
    end_time = timer()
    print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")
    
    scores = m.evaluate(dataset[l:r][:,:,21+classSize:], dataset[l:r][:, 0, 21 : (21+classSize)])
    print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]) + ' fold ' + str(fold))
    
    
    m.save('model_kfold' + str(fold) + '.h5')  # creates a HDF5 file 
    with open('model_scores_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(scores, file_pi)
    with open('model_history_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    valid_indexes.append((l, r))
    r += 113350
    l += 113350

with open('model_kfold_indexes', 'wb') as file_pi:
    pickle.dump(valid_indexes, file_pi)   

1

Hyper Parameters

Learning Rate: 0.0005
Drop out: 0.3
Batch dim: 64
Number of epochs: 5
Regularizers: 1e-04

Loss: categorical_crossentropy

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputWindow (InputLayer)     (None, 9, 21)             0         
_________________________________________________________________
conv1d_156 (Conv1D)          (None, 9, 64)             4096      
_________________________________________________________________
BN1 (BatchNormalization)     (None, 9, 64)             256       
_________________________________________________________________
conv1d_157 (Conv1D)          (None, 9, 64)             12352     
_________________________________________________________________
BN2 (BatchNormalization)     (None, 9, 64)             256       
_________________________________________________________________
conv1d_158 (Conv1D)          (None, 9, 64)             12352    

In [ ]:
kf = KFold(n_splits=10, random_state=10, shuffle=True)
fold = 0
valid_indexes = [([],[])]
for train, test in kf.split(dataset):

    fold += 1
    print(fold)
    
    m = get_model()
    start_time = timer()
    history = m.fit(dataset[train][:,:,0:21], dataset[train][:,:,21 : (21+classSize)], epochs=nn_epochs,
                    batch_size=batch_dim, validation_data=(dataset[test][:,:,0:21], dataset[test][:,:,21 : (21+classSize)]),shuffle=True)
    end_time = timer()
    print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")

    
    scores = m.evaluate(dataset[test][:,:,0:21], dataset[test][:,:,21 : (21+classSize)])
    print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]) + ' fold ' + str(fold))
    
    
    m.save('model_kfold' + str(fold) + '.h5')  # creates a HDF5 file 
    with open('model_scores_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(scores, file_pi)
    with open('model_history_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    valid_indexes.append((train, test))

with open('model_kfold_indexes', 'wb') as file_pi:
    pickle.dump(valid_indexes, file_pi)    
    